In [1]:
import numpy as np 
from copy import deepcopy
from utils import slice_dataframe
from helper import load_valid_norm_datasets_sorted_by_date
from models import anomaly_classification
from tqdm import tqdm

# Load raw dataset

In [2]:
# 데이터를 로드 합니다.
_, norm_target_df, scalers = load_valid_norm_datasets_sorted_by_date('./pu_batt_sample.csv')

#  Generate trainable datasets
- LSTM 모델에 작동 가능하도록 전처리 합니다.
- 전처리 목록
  1. N, N_col -> N/10, 10, N_col 형태로 변환합니다. (RNN 계열 데이터 포맷)
  2. Anomaly data을 생성해 기존 정상 데이터 셋에 추가합니다. (정상 데이터 평균 1, 분산 1, 평균이 3이고 분산이 2인 anomaly data 추가)
  3. 분류 데이터를 생성합니다.
  4. 데이터를 섞습니다.

In [3]:
# 데이터 일정한 크기와 간격으로 slice 합니다.
steps = 10
strides=10
n_sample=None
sliced_dfs = slice_dataframe(norm_target_df.iloc[:n_sample], interval=steps, stride=strides, output_type='numpy')
print(np.mean(np.array(sliced_dfs)))

ano_sliced_dfs = deepcopy(sliced_dfs)
# anomlay dataset 을 생성합니다.
for idx, sliced_df in enumerate(tqdm(ano_sliced_dfs)):
    n_anomaly = np.random.randint(1, 100)  # outlier dataset 생성
    index = []
    rand_values = [] 
    # 데이터셋 내 random 한 위치와 random 한 개수로 anomaly dataset 삽입
    for _ in range(n_anomaly):
        timestep_idx = np.random.randint(0, 10)
        col_idx = np.random.randint(0, 100)
        index.append([timestep_idx, col_idx])
    # anomlay value 생성
    anomaly_mean, anomaly_std = 3, 2
    rand_values = np.random.normal(anomaly_mean, anomaly_std, n_anomaly)
    index = np.array(index)
    sliced_df[index[:, 0], index[:, 1]] = rand_values
    ano_sliced_dfs[idx] = sliced_df 
print(np.mean(ano_sliced_dfs), np.mean(sliced_dfs))

100%|██████████| 197548/197548 [00:32<00:00, 6093.80it/s]


0.6628396571549711


100%|██████████| 197548/197548 [00:21<00:00, 9069.72it/s]


0.7759533069243235 0.6628396571549711


In [4]:
# train_xs, train_ys
train_xs = np.array(ano_sliced_dfs + sliced_dfs)
train_ys = np.array([0]*len(ano_sliced_dfs) + [1]*len(sliced_dfs))
del(ano_sliced_dfs)
del(sliced_dfs)

# shuffle 
shuffle_idx = np.arange(len(train_ys)) 
np.random.shuffle(shuffle_idx)
train_xs = train_xs[shuffle_idx]
train_ys = train_ys[shuffle_idx]

In [8]:
np.save('train_xs.npy', train_xs)
np.save('train_ys.npy', train_ys)

In [39]:
# 예제 입력 및 출력 차원 및 은닉 유닛 수
input_n_features = norm_target_df.shape[-1]
hidden_units = 64

# 모델 생성
model = anomaly_classification(steps, input_n_features, hidden_units)

# 모델 요약
model.summary()
model.compile('adam', loss='sparse_categorical_crossentropy')
model.save('./tmp.h5')

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 10, 100)]         0         
                                                                 
 lstm_3 (LSTM)               (None, 64)                42240     
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                                 
 dense_11 (Dense)            (None, 2)                 130       
                                                                 
Total params: 50690 (198.01 KB)
Trainable params: 50690 (198.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


/Users/kimseongjung/anaconda3/envs/rice-detector_py3_9/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [27]:
model.fit(train_xs, train_ys, epochs=1000)

Epoch 1/1000
2/2 [==============================] - 0s 4ms/step - loss: 2.9076e-05
Epoch 2/1000
2/2 [==============================] - 0s 4ms/step - loss: 2.8994e-05
Epoch 3/1000
2/2 [==============================] - 0s 4ms/step - loss: 2.8859e-05
Epoch 4/1000
2/2 [==============================] - 0s 3ms/step - loss: 2.8763e-05
Epoch 5/1000
2/2 [==============================] - 0s 3ms/step - loss: 2.8640e-05
Epoch 6/1000
2/2 [==============================] - 0s 4ms/step - loss: 2.8536e-05
Epoch 7/1000
2/2 [==============================] - 0s 4ms/step - loss: 2.8417e-05
Epoch 8/1000
2/2 [==============================] - 0s 4ms/step - loss: 2.8323e-05
Epoch 9/1000
2/2 [==============================] - 0s 4ms/step - loss: 2.8188e-05
Epoch 10/1000
2/2 [==============================] - 0s 4ms/step - loss: 2.8098e-05
Epoch 11/1000
2/2 [==============================] - 0s 4ms/step - loss: 2.7963e-05
Epoch 12/1000
2/2 [==============================] - 0s 4ms/step - loss: 2.7871e-05
E

In [38]:
y_hat = model.predict(train_xs)
y_hat = np.argmax(y_hat, axis=-1)
print('Accuracy: {}'.format(np.mean(y_hat == train_ys) * 100))

2/2 [==============================] - 0s 2ms/step
Accuracy: 100.0
